# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Data import
Since translating the whole language of English to French will take lots of time to train, only a small portion of the English corpus is imported.

In [2]:
import helper
import problem_unittests as tests

# Load the data
source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Data exploration

In [3]:
view_sentence_range = (10, 14)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 10 to 14:
the lime is her least liked fruit , but the banana is my least liked .
he saw a old yellow truck .
india is rainy during june , and it is sometimes warm in november .
that cat was my most loved animal .

French sentences 10 to 14:
la chaux est son moins aimé des fruits , mais la banane est mon moins aimé.
il a vu un vieux camion jaune .
inde est pluvieux en juin , et il est parfois chaud en novembre .
ce chat était mon animal le plus aimé .


## Preprocessing Function
### Text to Word Ids
Turn words into ids for processing. After each sentence from `target_text` the id `<EOS>` is added to help the NN to predict when the sentence should end.

In [5]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):

    # Implement Function
    source_sentences = [sentence for sentence in source_text.split('\n')]
    # Add <EOS> at the end of each target sentence
    target_sentences = [sentence + ' <EOS>' for sentence in target_text.split('\n')]
    source_id_text= [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_sentences]
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()] for sentence in target_sentences]
    
    return source_id_text, target_id_text

# Helper: Function test
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Preprocess all the data and save it to file with the helper function.

In [6]:
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point

In [7]:
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### TensorFlow version check and access to GPU

In [9]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) in [LooseVersion('1.0.0'), LooseVersion('1.0.1')], 'This project requires TensorFlow version 1.0  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('GPU found! Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
GPU found! Default GPU Device: /gpu:0


## Build the Neural Network
Build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoding_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Create TF Placeholders for the Neural Network:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.

In [10]:
def model_inputs():

    # Placeholders
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None])
    lr = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return input_data, targets, lr, keep_prob

# Helper: Function test 
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoding Input
Function `process_decoding_input` to remove the last word id from each batch in `target_data` and concat the GO ID to the beginning of each batch.

In [11]:
def process_decoding_input(target_data, target_vocab_to_int, batch_size):

    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1,1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending],1)
    
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_decoding_input(process_decoding_input)

Tests Passed


### Encoding
`encoding_layer()` to create a Encoder RNN layer using [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn).

In [12]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):

    # Encoding layer
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    MultiCell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    enc_output, enc_state = tf.nn.dynamic_rnn(MultiCell, rnn_inputs, dtype=tf.float32)
    
    return enc_state

# Helper: Function test
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Training logits using [`tf.contrib.seq2seq.simple_decoder_fn_train()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_train) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder). 

In [15]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):

    # Decoding layer train
    train_decoder = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, train_decoder, dec_embed_input, sequence_length, 
                                                             scope = decoding_scope)
    # Train logits
    train_logits = tf.nn.dropout(train_pred, keep_prob=keep_prob)
    train_logits = output_fn(train_logits)
    
    return train_logits


# Helper: Function test
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Inference logits using [`tf.contrib.seq2seq.simple_decoder_fn_inference()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_inference) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder). 

In [14]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    
    # Decoding layer inference
    num_decoder_symbols = vocab_size
    infer_decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_inference(output_fn, encoder_state, dec_embeddings,
                                                                     start_of_sequence_id, end_of_sequence_id,
                                                                     maximum_length, num_decoder_symbols)
    # Inference logits
    inference_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, infer_decoder_fn, scope=decoding_scope)
    
    return inference_logits


# Helper: Function test
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
`decoding_layer()` to create a Decoder RNN layer.

- Create RNN cell for decoding using `rnn_size` and `num_layers`.
- Create the output fuction using [`lambda`](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to transform it's input, logits, to class logits.
- Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)` function to get the training logits.
- Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob)` function to get the inference logits.

Share variables between training and inference with [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope).

In [16]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, target_vocab_to_int, keep_prob):

    # Decoding layer
    start_of_sequence_id = target_vocab_to_int['<GO>']
    end_of_sequence_id = target_vocab_to_int['<EOS>']
    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm]*num_layers)
    cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    
    
    # Output function with lambda operator
    output_fn = lambda x: tf.contrib.layers.fully_connected(x, vocab_size, None, scope=decoding_scope) 
    
    # Share the variables
    with tf.variable_scope('decoding') as decoding_scope:    
        training_logits = decoding_layer_train(encoder_state, cell, dec_embed_input, sequence_length, 
                                                decoding_scope, output_fn, keep_prob)
        
    decoding_scope.reuse_variables()

    inference_logits = decoding_layer_infer(encoder_state, cell, dec_embeddings, start_of_sequence_id, 
                                                end_of_sequence_id, sequence_length, 
                                                vocab_size, decoding_scope, output_fn, keep_prob)   
        
    return training_logits, inference_logits


# Helper: Function test
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Build the pipeline:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob)`.
- Process target data using your `process_decoding_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)`.

In [18]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, target_vocab_to_int):

    # Sequence 2 sequence NN pipeline
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size)
    encoder_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob)
    
    decoding_input = process_decoding_input(target_data, target_vocab_to_int, batch_size)
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, dec_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, decoding_input)
    
    train, infer = decoding_layer(dec_embed_input, dec_embeddings, encoder_state, target_vocab_size, sequence_length, 
                                   rnn_size, num_layers, target_vocab_to_int, keep_prob)
    return (train, infer)


# Helper: Function test
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability

In [19]:
# Number of epochs
epochs = 5
# Batch size
batch_size = 512
# RNN size
rnn_size = 130
# Number of layers
num_layers = 2
# Embedding size
encoding_embedding_size = 200
decoding_embedding_size = 200
# Learning rate
learning_rate = 0.01
# Dropout keep probability
keep_probability = 0.5

### Build the Graph
Build the graph using the neural network.

In [20]:

save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_source_sentence_length = max([len(sentence) for sentence in source_int_text])

# Graph build-up
train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob = model_inputs()
    sequence_length = tf.placeholder_with_default(max_source_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)
    
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), len(target_vocab_to_int),
        encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, target_vocab_to_int)

    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

### Train
Network training.

In [23]:
import time

def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1]), (0,0)],
            'constant')

    return np.mean(np.equal(target, np.argmax(logits, 2)))

train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:batch_size])

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                helper.batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 sequence_length: target_batch.shape[1],
                 keep_prob: keep_probability})
            
            batch_train_logits = sess.run(
                inference_logits,
                {input_data: source_batch, keep_prob: 1.0})
            batch_valid_logits = sess.run(
                inference_logits,
                {input_data: valid_source, keep_prob: 1.0})
                
            train_acc = get_accuracy(target_batch, batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target), batch_valid_logits)
            end_time = time.time()
            if batch_i % 10 == 0:
                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    0/269 - Train Accuracy:  0.242, Validation Accuracy:  0.310, Loss:  5.873
Epoch   0 Batch   10/269 - Train Accuracy:  0.292, Validation Accuracy:  0.359, Loss:  3.446
Epoch   0 Batch   20/269 - Train Accuracy:  0.363, Validation Accuracy:  0.419, Loss:  2.901
Epoch   0 Batch   30/269 - Train Accuracy:  0.438, Validation Accuracy:  0.468, Loss:  2.562
Epoch   0 Batch   40/269 - Train Accuracy:  0.453, Validation Accuracy:  0.496, Loss:  2.399
Epoch   0 Batch   50/269 - Train Accuracy:  0.470, Validation Accuracy:  0.514, Loss:  2.117
Epoch   0 Batch   60/269 - Train Accuracy:  0.482, Validation Accuracy:  0.491, Loss:  1.729
Epoch   0 Batch   70/269 - Train Accuracy:  0.430, Validation Accuracy:  0.440, Loss:  1.512
Epoch   0 Batch   80/269 - Train Accuracy:  0.432, Validation Accuracy:  0.439, Loss:  1.297
Epoch   0 Batch   90/269 - Train Accuracy:  0.415, Validation Accuracy:  0.473, Loss:  1.269
Epoch   0 Batch  100/269 - Train Accuracy:  0.452, Validation Accuracy

Epoch   3 Batch   80/269 - Train Accuracy:  0.875, Validation Accuracy:  0.872, Loss:  0.200
Epoch   3 Batch   90/269 - Train Accuracy:  0.869, Validation Accuracy:  0.881, Loss:  0.223
Epoch   3 Batch  100/269 - Train Accuracy:  0.893, Validation Accuracy:  0.875, Loss:  0.203
Epoch   3 Batch  110/269 - Train Accuracy:  0.878, Validation Accuracy:  0.879, Loss:  0.193
Epoch   3 Batch  120/269 - Train Accuracy:  0.876, Validation Accuracy:  0.879, Loss:  0.202
Epoch   3 Batch  130/269 - Train Accuracy:  0.893, Validation Accuracy:  0.881, Loss:  0.212
Epoch   3 Batch  140/269 - Train Accuracy:  0.874, Validation Accuracy:  0.881, Loss:  0.210
Epoch   3 Batch  150/269 - Train Accuracy:  0.877, Validation Accuracy:  0.890, Loss:  0.194
Epoch   3 Batch  160/269 - Train Accuracy:  0.875, Validation Accuracy:  0.877, Loss:  0.198
Epoch   3 Batch  170/269 - Train Accuracy:  0.888, Validation Accuracy:  0.894, Loss:  0.194
Epoch   3 Batch  180/269 - Train Accuracy:  0.905, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [24]:
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [25]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, first it needs to be preprocessed. Therefore the function `sentence_to_seq()` is implemented.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
- Convert words not in the vocabulary, to the `<UNK>` word id.

In [26]:
def sentence_to_seq(sentence, vocab_to_int):

    # Sentence to sequence
    word_list = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.lower().split()]
    
    return word_list

#Helper: Function test
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [27]:
translate_sentence = 'he saw a yellow bird in summer .'

translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('logits:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

Input
  Word Ids:      [5, 9, 10, 184, 205, 63, 208, 88]
  English Words: ['he', 'saw', 'a', 'yellow', 'bird', 'in', 'summer', '.']

Prediction
  Word Ids:      [226, 7, 46, 305, 237, 71, 1]
  French Words: ['il', 'a', 'vu', 'une', 'nouvelle', 'en', '<EOS>']


## Discussion: Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  Additionally, the translations in this data set were made by Google translate, so the translations themselves aren't particularly good.  (We apologize to the French speakers out there!) Thankfully, for this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.